In [1]:
using JuMP
using Hypatia
using LinearAlgebra
using Combinatorics

In [2]:
function isCLDUI(gammadelta::Vector{Int64})
    n = Int64(length(gammadelta)/4)
    for i in 1:n
        if gammadelta[i]+gammadelta[3*n+i]!=gammadelta[n+i]+gammadelta[2*n+i]
            return false
        end
    end
    return true
end

function isLDOI(gammadelta::Vector{Int64})
    n = Int64(length(gammadelta)/4)
    for i in 1:n
        if (gammadelta[i]+gammadelta[n+i]+gammadelta[2*n+i]+gammadelta[3*n+i])%2!=0
            return false
        end
    end
    return true
end

function ListGammaDeltaHalfCLDUI(n::Int,t::Int)
    L = [x for x in ListGammaDeltaHalf(n,t) if isCLDUI(x)]
    return L 
end

function ListGammaDeltaOtherHalfCLDUI(n::Int,t::Int)
    L = [x for x in ListGammaDeltaOtherHalf(n,t) if isCLDUI(x)]
    return L 
end

function ListGammaDeltaHalfLDOI(n::Int,t::Int)
    L = [x for x in ListGammaDeltaHalf(n,t) if isLDOI(x)]
    return L 
end

function ListGammaDeltaOtherHalfLDOI(n::Int,t::Int)
    L = [x for x in ListGammaDeltaOtherHalf(n,t) if isLDOI(x)]
    return L 
end

function ListGammaDeltaHalf(n::Int,t::Int)
    L = Vector{Vector{Int}}()
    for i in 1:n
        for j in i+1:n
            for k in 1:binomial(n+t-1,t)
                for l in 1:binomial(n+t-1,t)
                    push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, 1))[j],collect(multiexponents(n, t))[k],collect(multiexponents(n, t))[l]))    
                end
            end
        end 
    end
    for i in 1:n
        
        for k in 1:binomial(n+t-1,t)
            for l in k+1:binomial(n+t-1,t)
                push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, 1))[i],collect(multiexponents(n, t))[k],collect(multiexponents(n, t))[l]))    
            end
        end
        
    end
    return L
end

function ListGammaDeltaOtherHalf(n::Int,t::Int)
    L = Vector{Vector{Int}}()
    for i in 1:n
        for j in i+1:n
            for k in 1:binomial(n+t-1,t)
                for l in 1:binomial(n+t-1,t)
                    push!(L,vcat(collect(multiexponents(n, 1))[j],collect(multiexponents(n, 1))[i],collect(multiexponents(n, t))[l],collect(multiexponents(n, t))[k]))    
                end
            end
        end 
    end
    for i in 1:n
        
        for k in 1:binomial(n+t-1,t)
            for l in k+1:binomial(n+t-1,t)
                push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, 1))[i],collect(multiexponents(n, t))[l],collect(multiexponents(n, t))[k]))    
            end
        end
        
    end
    return L
end
    
function ListGammaDeltaDiag(n::Int,t::Int)
    L = Vector{Vector{Int}}()
    for i in 1:n
        for k in 1:binomial(n+t-1,t)
            push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, 1))[i],collect(multiexponents(n, t))[k],collect(multiexponents(n, t))[k]))    
        end  
    end
    return L
end

#List of all relevant  gamma,gamma',delta,delta'
function ListGammaDelta(n::Int,t::Int)# gamma = 1, gamma' = 1, delta = t, delta' = t' 
    #L = []
    L = Vector{Vector{Int}}()
    for i in 1:n
        for j in 1:n
            for k in 1:binomial(n+t-1,t)
                for l in 1:binomial(n+t-1,t)
                    push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, 1))[j],collect(multiexponents(n, t))[k],collect(multiexponents(n, t))[l]))    
                end
            end
        end 
    end
    return L
end

#Indexing of moment matrix B with monomials. Full range, no block structure
function IndexListB(n::Int,t::Int)
    L = []
    #alpha = 1, alpha' = 0
    for i in 1:n #selects the possible entries of alpha
        for s in 0:t #beta = t-s, beta' = s
            for j in 1:binomial(n+t-s-1,t-s) #j and k iterate over the amount of monomials for beta and beta'
                for k in 1:binomial(n+s+-1,s)
                    push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, t-s))[j],collect(multiexponents(n, s))[k]))
                end 
            end  
        end
    end
    return L
end 

#Computes given gammadelta the corresponding positions in moment matrix B, where a 1 has to be placed
function BGammaDelta(gamma::Vector{Int64},gamma2::Vector{Int64},delta::Vector{Int64},delta2::Vector{Int64})
    n = length(gamma)
    t = sum(delta[i] for i in 1:n)
    IndexB = IndexListB(n,t)
    B = zeros(Int64,length(IndexB),length(IndexB))
    for i in 1:length(IndexB) #iterates over rows of B
        if all([gamma[j]-IndexB[i][j] for j in 1:n] .≥ 0) && all([delta[j]-IndexB[i][j+n] for j in 1:n] .≥ 0) && all([delta2[j]-IndexB[i][j+2*n] for j in 1:n] .≥ 0)#check if 1 in this row
            B[i,findfirst(==(vcat(gamma2,[delta2[j]-IndexB[i][j+2*n] for j in 1:n],[delta[j]-IndexB[i][j+n] for j in 1:n])), IndexB)] = 1
        end
    end
    return B
end

#computes the monomials Indexing I_1,s
#a,b,b' all vectors in N^n. weight a is 1, weight of b is t+s/2, weight of b' is t-s/2
function BigIndexList(n::Int,t::Int,s::Int)#-t<=s<=t and s same parity as as t
    L = []
    for i in 1:n
        for j in 1:binomial(n+Int((t+s)/2)-1,Int((t+s)/2))
            for k in 1:binomial(n+Int((t-s)/2)-1,Int((t-s)/2))
                push!(L,vcat(collect(multiexponents(n, 1))[i],collect(multiexponents(n, Int((t+s)/2)))[j],collect(multiexponents(n, Int((t-s)/2)))[k]))
            end
        end
    end
    return L
end 

#gives List of indices as natural numbers relevant for I_1,s
function subIndexListB(n::Int,t::Int,s::Int)
    I = []
    IndexB = IndexListB(n,t)
    IndexBlock = BigIndexList(n,t,s)
    for i in 1:length(IndexBlock)
        push!(I,findfirst(==(IndexBlock[i]), IndexB))
    end
    return I
end

#List of indices as monomials indexing CLDUI blocks for I_1,s
function CLDUIBlockIndexList(n::Int,t::Int,s::Int)
    L = []
    B = BigIndexList(n,t,s)
    push!(L,[])
    push!(L[1],B[1])
    for i in 2:length(B)
        appended = false
        j = 1
        while appended == false && j in 1:length(L)
            if [L[j][1][n+k] for k in 1:n]-[L[j][1][2*n+k] for k in 1:n]-[L[j][1][k] for k in 1:n] == [B[i][n+k] for k in 1:n]-[B[i][2*n+k] for k in 1:n]-[B[i][k] for k in 1:n] 
                appended = true
                push!(L[j],B[i])
            end
            j = j+1
        end
        if appended == false
            push!(L,[])
            push!(L[length(L)],B[i])
        end
    end
    return L
end

function subIndexListBCLDUI(n::Int,t::Int,s::Int,j::Int)
    I = []
    IndexB = IndexListB(n,t)
    IndexBlock = CLDUIBlockIndexList(n,t,s)[j]
    for i in 1:length(IndexBlock)
        push!(I,findfirst(==(IndexBlock[i]), IndexB))
    end
    return I
end

#List of indices as monomials indexing CLDUI blocks for I_1,s
function LDOIBlockIndexList(n::Int,t::Int,s::Int)
    L = []
    B = BigIndexList(n,t,s)
    push!(L,[])
    push!(L[1],B[1])
    for i in 2:length(B) #iterates over entries of B to decide to which Clique every entry goes
        appended = false
        j = 1 #iterates over all the created drawers for the entries of B
        while appended == false && j in 1:length(L)
            #create gammadelta based on i and j:
            ab = B[i]
            abprime = L[j][1]
            deltagammasum = ab[1:n] + ab[n+1:2*n] + ab[2*n+1:3*n] + abprime[1:n] + abprime[n+1:2*n] + abprime[2*n+1:3*n]
            if all(x -> x % 2 == 0, deltagammasum)
                appended = true
                push!(L[j],B[i])
            end
            j = j+1
        end
        if appended == false
            push!(L,[])
            push!(L[length(L)],B[i])
        end
    end
    return L
end

function subIndexListBLDOI(n::Int,t::Int,s::Int,j::Int)
    I = []
    IndexB = IndexListB(n,t)
    IndexBlock = LDOIBlockIndexList(n,t,s)[j]
    for i in 1:length(IndexBlock)
        push!(I,findfirst(==(IndexBlock[i]), IndexB))
    end
    return I
end

function GammaDeltaAtij(n::Int,t::Int,i::Int,j::Int)
    ab = IndexListB(n,t)[i]
    abprime = IndexListB(n,t)[j]
    return vcat([ab[k] for k in 1:n],[abprime[k] for k in 1:n],[ab[n+k] + abprime[2*n+k] for k in 1:n],[ab[2*n+k] + abprime[n+k] for k in 1:n])
end

function e(i::Int,n::Int)
    return [h == i ? 1 : 0 for h in 1:n]
end

function multinomCoef(n::Int,v::Vector{Int64},t::Int)#at the place helper function is called: m index in the List collect(multiexponents(n,t-1))
    return factorial(t)//prod(factorial(v[o]) for o in 1:n)
end


multinomCoef (generic function with 1 method)

In [3]:
function DPS(rho::LinearAlgebra.Hermitian,t::Int)
    model = Model(Hypatia.Optimizer)
    n = Int(sqrt(size(rho)[1]))

    time = @elapsed begin
    @variable(model, z[1:length(ListGammaDeltaDiag(n,t))+length(ListGammaDeltaHalf(n,t))])
    end
    println("Time taken defining the variables: ", time, " seconds")

    LVariablesHalf = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalf(n,t))
    lVarH = length(LVariablesHalf)
    LVariablesFull = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalf(n,t),ListGammaDeltaOtherHalf(n,t))
    lVarF = length(LVariablesFull)

    LIndex = IndexListB(n,t)
    lIndex = length(LIndex)

    time = @elapsed begin
    B = Matrix{VariableRef}(undef, lIndex, lIndex)
    for s in t:-2:-t
        LijTempS = subIndexListB(n,t,s)
        ltemp = length(LijTempS)
        for i_idx in 1:ltemp, j_idx in i_idx:ltemp
            i = LijTempS[i_idx]
            j = LijTempS[j_idx]
            GammaDelta = GammaDeltaAtij(n,t,i,j)
            zPos = findfirst(==(GammaDelta),LVariablesFull)
            if zPos > lVarH # makes sure to use the correct half of the symmetric z
                zPos = zPos - (lVarF - lVarH)
            end
            B[i,j] = z[zPos]
            B[j,i] = z[zPos]
        end
    end
    end
    println("Time taken setting up the full moment matrix: ", time, " seconds")
    
    time = @elapsed begin
    for s in t:-2:-t
        @constraint(model, LinearAlgebra.Hermitian(B[subIndexListB(n,t,s),subIndexListB(n,t,s)]) in HermitianPSDCone())
    end
    end
    println("Time taken defining the psd blocks: ", time, " seconds")
    

    time = @elapsed begin
    Lexp = collect(multiexponents(n,t-1))
    for i in 1:n, j in 1:n, k in 1:n, l in 1:n
        if i==k && j ==l #look up in diagonl part
            @constraint(model, sum(z[findfirst(==(vcat(e(i,n),e(k,n),e(j,n) + Lexp[m],e(l,n) + Lexp[m])), ListGammaDeltaDiag(n,t))]*multinomCoef(n,Lexp[m],t-1) for m in 1:length(Lexp)) == rho[(i-1)*n+j,(k-1)*n+l])
        else #look up in the other half
            temp = 0
            for m in 1:length(Lexp)
                if vcat(e(i,n),e(k,n),e(j,n) + Lexp[m],e(l,n) + Lexp[m]) in Set(ListGammaDeltaHalf(n,t))
                    temp = temp + z[length(ListGammaDeltaDiag(n,t)) + findfirst(==(vcat(e(i,n),e(k,n),e(j,n) + Lexp[m],e(l,n) + Lexp[m])), ListGammaDeltaHalf(n,t))]*multinomCoef(n,Lexp[m],t-1)
                else
                    temp = temp + z[length(ListGammaDeltaDiag(n,t)) + findfirst(==(vcat(e(i,n),e(k,n),e(j,n) + Lexp[m],e(l,n) + Lexp[m])), ListGammaDeltaOtherHalf(n,t))]*multinomCoef(n,Lexp[m],t-1)
                end
            end
            @constraint(model,temp==rho[(i-1)*n+j,(k-1)*n+l])
        end
    end
    end
    println("Time taken setting up linear constraints: ", time, " seconds")
    
    Lexp = collect(multiexponents(n, t))
    @objective(model, Min, sum(sum(multinomCoef(n,Lexp[j],t)*z[findfirst(==(vcat(e(i,n),e(i,n),Lexp[j],Lexp[j])), ListGammaDeltaDiag(n,t))] for j in 1:length(Lexp)) for i in 1:n))
    
    optimize!(model)
    objective_value(model)
end

function CLDUI_DPS(rho::LinearAlgebra.Hermitian,t::Int)
    model = Model(Hypatia.Optimizer)
    
    n = Int(sqrt(size(rho)[1]))

    time = @elapsed begin
    LVariablesHalf = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalfCLDUI(n,t))
    lVarH = length(LVariablesHalf)
    LVariablesFull = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalfCLDUI(n,t),ListGammaDeltaOtherHalfCLDUI(n,t))
    lVarF = length(LVariablesFull)
    @variable(model, z[1:lVarH])
    end
    println("Time taken defining the variables: ", time, " seconds")

    LIndex = IndexListB(n,t)
    lIndex = length(LIndex)

    time = @elapsed begin
    B = Matrix{VariableRef}(undef, lIndex, lIndex)
    for s in t:-2:-t, j in 1:length(CLDUIBlockIndexList(n,t,s))
        LijTempS = subIndexListBCLDUI(n,t,s,j)
        ltemp = length(LijTempS)
        for i_idx in 1:ltemp, j_idx in i_idx:ltemp
            i = LijTempS[i_idx]
            j = LijTempS[j_idx]
            GammaDelta = GammaDeltaAtij(n,t,i,j)
            zPos = findfirst(==(GammaDelta),LVariablesFull)
            if zPos > lVarH # makes sure to use the correct half of the symmetric z
                zPos = zPos - (lVarF - lVarH)
            end
            B[i,j] = z[zPos]
            B[j,i] = z[zPos]
        end
    end
    end
    println("Time taken setting up the full moment matrix: ", time, " seconds")
    
    time = @elapsed begin
    for s in t:-2:-t, j in 1:length(CLDUIBlockIndexList(n,t,s))
        @constraint(model, LinearAlgebra.Hermitian(B[subIndexListBCLDUI(n,t,s,j),subIndexListBCLDUI(n,t,s,j)]) in HermitianPSDCone()) 
    end
    end
    println("Time taken defining the psd blocks: ", time, " seconds")

    time = @elapsed begin
    Lexp = collect(multiexponents(n, t-1))
    #(ii,jj) and (ij,ij) vs (ij,kl)
    for i in 1:n, j in 1:n #this is (ij,ij), therefore symmetric list is the correct one
        @constraint(model, sum(z[findfirst(==(vcat(e(i,n),e(i,n),e(j,n) + Lexp[m],e(j,n) + Lexp[m])), ListGammaDeltaDiag(n,t))]*multinomCoef(n,Lexp[m],t-1) for m in 1:length(Lexp)) == rho[(i-1)*n+j,(i-1)*n+j])  
    end

    for i in 1:n, k in 1:n # now (ii,kk)
        if i!=k
            temp = 0
            for m in 1:length(Lexp)
                if vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m]) in Set(ListGammaDeltaHalfCLDUI(n,t))
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m])), ListGammaDeltaHalfCLDUI(n,t))]*multinomCoef(n,Lexp[m],t-1)
                else
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m])), ListGammaDeltaOtherHalfCLDUI(n,t))]*multinomCoef(n,Lexp[m],t-1) 
                end
            end
            @constraint(model,temp==rho[(i-1)*n+i,(k-1)*n+k])
        end
    end
    end
    println("Time taken setting up linear constraints: ", time, " seconds")
    Lexp = collect(multiexponents(n, t))
    @objective(model, Min, sum(sum(multinomCoef(n,Lexp[j],t)*z[findfirst(==(vcat(e(i,n),e(i,n),Lexp[j],Lexp[j])), ListGammaDeltaDiag(n,t))] for j in 1:length(Lexp)) for i in 1:n))

    optimize!(model)
    objective_value(model)
end

function LDOI_DPS(rho::LinearAlgebra.Hermitian,t::Int)
    model = Model(Hypatia.Optimizer)
    
    n = Int(sqrt(size(rho)[1]))

    time = @elapsed begin
    LVariablesHalf = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalfLDOI(n,t))
    lVarH = length(LVariablesHalf)
    LVariablesFull = vcat(ListGammaDeltaDiag(n,t),ListGammaDeltaHalfLDOI(n,t),ListGammaDeltaOtherHalfLDOI(n,t))
    lVarF = length(LVariablesFull)
    @variable(model, z[1:lVarH])
    end
    println("Time taken defining the variables: ", time, " seconds")

    LIndex = IndexListB(n,t)
    lIndex = length(LIndex)

    time = @elapsed begin
    B = Matrix{VariableRef}(undef, lIndex, lIndex)
    for s in t:-2:-t, j in 1:length(LDOIBlockIndexList(n,t,s))
        LijTempS = subIndexListBLDOI(n,t,s,j)
        ltemp = length(LijTempS)
        for i_idx in 1:ltemp, j_idx in i_idx:ltemp 
            i = LijTempS[i_idx]
            j = LijTempS[j_idx]
            GammaDelta = GammaDeltaAtij(n,t,i,j)
            zPos = findfirst(==(GammaDelta),LVariablesFull)
            if zPos > lVarH # makes sure to use the correct half of the symmetric z
                zPos = zPos - (lVarF - lVarH)
            end
            B[i,j] = z[zPos]
            B[j,i] = z[zPos]
        end
    end
    end
    println("Time taken setting up the full moment matrix: ", time, " seconds")
    
    time = @elapsed begin
    for s in t:-2:-t, j in 1:length(LDOIBlockIndexList(n,t,s))
        @constraint(model, LinearAlgebra.Hermitian(B[subIndexListBLDOI(n,t,s,j),subIndexListBLDOI(n,t,s,j)]) in HermitianPSDCone())
    end
    end
    println("Time taken defining the psd blocks: ", time, " seconds")

    time = @elapsed begin
    Lexp = collect(multiexponents(n,t-1))
    #(ii,jj) and (ij,ij) and (ij,ji) vs (ij,kl)
    for i in 1:n, j in 1:n #this is (ij,ij), therefore symmetric list is correct here
        @constraint(model, sum(z[findfirst(==(vcat(e(i,n),e(i,n),e(j,n) + Lexp[m],e(j,n) + Lexp[m])), ListGammaDeltaDiag(n,t))]*multinomCoef(n,Lexp[m],t-1) for m in 1:length(Lexp)) == rho[(i-1)*n+j,(i-1)*n+j])
    end

    for i in 1:n, k in 1:n# now (ii,kk)
        if i!=k
            temp = 0
            for m in 1:length(Lexp)
                if vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m]) in Set(ListGammaDeltaHalfLDOI(n,t))
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m])), ListGammaDeltaHalfLDOI(n,t))]*multinomCoef(n,Lexp[m],t-1)
                else
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(i,n) + Lexp[m],e(k,n) + Lexp[m])), ListGammaDeltaOtherHalfLDOI(n,t))]*multinomCoef(n,Lexp[m],t-1)
                end
            end
            @constraint(model,temp==rho[(i-1)*n+i,(k-1)*n+k])
        end
    end

    for i in 1:n, k in 1:n # now (ik,ki)
        if i!=k
            temp = 0
            for m in 1:length(Lexp)
                if vcat(e(i,n),e(k,n),e(k,n) + Lexp[m],e(i,n) + Lexp[m]) in Set(ListGammaDeltaHalfLDOI(n,t))
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(k,n) + Lexp[m],e(i,n) + Lexp[m])), ListGammaDeltaHalfLDOI(n,t))]*multinomCoef(n,Lexp[m],t-1)
                else
                    temp = temp + z[length(ListGammaDeltaDiag(n,t))+findfirst(==(vcat(e(i,n),e(k,n),e(k,n) + Lexp[m],e(i,n) + Lexp[m])), ListGammaDeltaOtherHalfLDOI(n,t))]*multinomCoef(n,Lexp[m],t-1)
                end
            end
            @constraint(model,temp==rho[(i-1)*n+k,(k-1)*n+i])
        end
    end
    end
    println("Time taken setting up linear constraints: ", time, " seconds")
    
    @objective(model, Min, sum(sum(factorial(t)//prod(factorial(collect(multiexponents(n, t))[j][o]) for o in 1:n)*z[findfirst(==(vcat([h == i ? 1 : 0 for h in 1:n],[h == i ? 1 : 0 for h in 1:n],collect(multiexponents(n, t))[j],collect(multiexponents(n, t))[j])), ListGammaDeltaDiag(n,t))] for j in 1:length(collect(multiexponents(n, t)))) for i in 1:n))

    
    optimize!(model)
    objective_value(model)
end

LDOI_DPS (generic function with 1 method)

In [ ]:
#test state

a = 2
b = 0.5


rho = LinearAlgebra.Hermitian([
1 0 0 0 1 0 0 0 1;
0 a 0 0 0 0 0 0 0;
0 0 b 0 0 0 0 0 0;
0 0 0 b 0 0 0 0 0;
1 0 0 0 1 0 0 0 1;
0 0 0 0 0 a 0 0 0;
0 0 0 0 0 0 a 0 0;
0 0 0 0 0 0 0 b 0;
1 0 0 0 1 0 0 0 1;
])# examples from paper

rho = LinearAlgebra.Hermitian(rho .* 1/tr(rho))#the implementation expects the input state to be normalized

CLDUI_DPS(rho,2)#first argument: state, second argument: level hierarchy

Time taken defining the variables: 0.000702 seconds
Time taken setting up the full moment matrix: 0.0231805 seconds
Time taken defining the psd blocks: 0.0187686 seconds
Time taken setting up linear constraints: 0.0100203 seconds
3 of 15 primal equality constraints are dependent

 iter        p_obj        d_obj |  abs_gap    x_feas    z_feas |      tau       kap        mu | dir_res     prox  step     alpha
    0   1.0000e+00  -1.0682e+00 | 6.30e+01  9.91e-01  1.03e+00 | 1.00e+00  1.00e+00  1.00e+00 |
    1   1.0000e+00  -5.0495e-02 | 4.43e+01  3.97e-01  4.12e-01 | 1.75e+00  3.12e-01  7.00e-01 | 4.4e-16  2.2e-01  co-a  3.00e-01
    2   1.0000e+00   7.9171e-01 | 2.18e+01  7.50e-02  7.77e-02 | 4.62e+00  1.11e-01  3.48e-01 | 6.7e-16  4.7e-01  co-a  5.00e-01
    3   1.0000e+00   9.5784e-01 | 6.46e+00  1.50e-02  1.55e-02 | 6.96e+00  2.88e-02  1.04e-01 | 4.4e-16  9.3e-01  co-a  7.00e-01
    4   1.0000e+00   9.6392e-01 | 9.84e-01  1.60e-02  1.66e-02 | 9.76e-01  1.31e-02  1.56e-02 | 8.8e-15  9.

1.5067773199247483e-30